In [1]:
# 导入包(PypI)
import re
import time
import random
import pymongo
import pandas as pd

from datetime import datetime
from tqdm import tqdm

from scrapy.selector import Selector

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

##### 启动 Chrome 浏览器实例：

打开 **terminal**, 下载 Chrome Driver (假的 Google Chrome)

```bash
brew install chromedriver
chmod +x /opt/homebrew/bin/chromedriver
```

输入以下命令（将 `your Chrome.exe path` 替换为您的 Google Chrome 浏览器路径）：
```bash
<your Chrome.exe path> --remote-debugging-port=9222 --user-data-dir="/Users/<your home folder name>/selenium/AutomationProfile"
```

- 请将your Chrome.exe path替换为您的Chrome浏览器所在路径，例如<br>`C:\Program Files\Google\Chrome\Application\chrome.exe`
- 配置 chromedriver 相关信息，请参考官方文档：[ChromeDriver](https://developer.chrome.com/docs/chromedriver)
- 来做个比方， 我的 *terminal command* 会是:

/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome \
  --remote-debugging-port=9222 \
  --user-data-dir="/Users/princess/selenium/AutomationProfile"

In [2]:
# 数据清洗可能需要使用的方法
def extract_number(List):
    list_cleaned = []
    for x in List:
        if x is None or not isinstance(x, str):
            list_cleaned.append(None)
            continue

        # Find all numeric patterns in the string
        numbers = re.findall(r'\d+\.?\d*', x)
        if numbers:
            # Join all found numbers (if multiple numbers are in one string)
            number = ''.join(numbers)
            list_cleaned.append(number)
        else:
            list_cleaned.append(None)

    return list_cleaned

def extract_large_number(List):
    list_cleaned = []
    for x in List:
        if x is None or not isinstance(x, str):
            list_cleaned.append(None)
            continue

        try:
            if '万' in x:
                number = x.replace('万', '')
                number = float(number) * 10000
                number = int(number)
            else:
                number = int(float(x))
            list_cleaned.append(number)
        except ValueError:
            # If conversion fails, append None or handle accordingly
            list_cleaned.append(None)

    return list_cleaned

def extract_date(List):
    list_cleaned = []
    for x in List:
        date_cleand = None  # Initialize with None
        if x is None or not isinstance(x, str):
            list_cleaned.append(date_cleand)
            continue

        match = re.search(r'(\d{4})-(\d{2})-(\d{2})', x)
        if match:
            date_cleand = match.group(0)
        else:
            match = re.search(r'(\d{2})-(\d{2})', x)
            if match:
                current_year = datetime.now().year
                month, day = match.groups()
                date_cleand = f'{current_year}-{month}-{day}'

        list_cleaned.append(date_cleand)

    return list_cleaned

In [3]:
# 配置Chrome浏览器
service = Service("/opt/homebrew/bin/chromedriver")

options = Options()
options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')# 远程调控模式启用
options.add_argument('--incognito')# 隐身/无痕模式启用
browser = webdriver.Chrome(service=service, options=options)
action = ActionChains(browser)

In [4]:
key_word = ""
num = 0

def check_login_status(browser):
    print("即将开始检查小红书登录状态...")
    print("爬取数据有账户封禁的风险，建议使用非主账号登录。")
    
    while True:
        page_source = browser.page_source
        if '登录探索更多内容' in page_source:
            print('暂未登录，请手动登录')
            print('检查时间:', time.ctime())
            time.sleep(10)
        else:
            print('登录成功')
            print('检查时间:', time.ctime())
            time.sleep(3)
            break

def check_page_load_status(browser, keyword):
    print("即将开始检查网页加载状态...")
    print("如果网页进入人机验证页面，请先手动完成验证。")
    
    while True:
        if keyword in browser.title:
            print('加载成功')
            print('检查时间:', time.ctime())
            break
        else:
            time.sleep(2)

def selenium_test():
    """
    登录状态检查，网页加载检查，根据用户输入进行搜索
    """
    global key_word, num
    browser.get('https://www.xiaohongshu.com/explore')
    
    check_login_status(browser)
    
    print("请在文本框中根据提示输入搜索关键词和笔记爬取数量。")
    keyword = input("搜索关键词：")
    try:
        num = int(input("笔记爬取数量："))
    except ValueError:
        print("请输入有效的整数作为爬取数量。")
        return
    
    url = f'https://www.xiaohongshu.com/search_result?keyword={keyword}&source=web_explore_feed'
    browser.get(url)
    time.sleep(3)

    check_page_load_status(browser, keyword)

selenium_test()

即将开始检查小红书登录状态...
爬取数据有账户封禁的风险，建议使用非主账号登录。
暂未登录，请手动登录
检查时间: Wed Dec  4 18:35:11 2024
暂未登录，请手动登录
检查时间: Wed Dec  4 18:35:21 2024
暂未登录，请手动登录
检查时间: Wed Dec  4 18:35:31 2024
登录成功
检查时间: Wed Dec  4 18:35:41 2024
请在文本框中根据提示输入搜索关键词和笔记爬取数量。
即将开始检查网页加载状态...
如果网页进入人机验证页面，请先手动完成验证。
加载成功
检查时间: Wed Dec  4 18:35:53 2024


In [5]:
def change_mode(browser):
    # 更改模式为图文
    try:
        mode_button = browser.find_element(By.XPATH, '//*[@id="search-type"]/div/div/div[2]')
        mode_button.click()
        print('已自动更改模式为图文。')
    except Exception as e:
        print(f"更改模式失败: {e}")

selected_order_text = ''
def change_sort_order(browser, action):
    # 更改排序方式
    sort_order = {
        '综合': 1,
        '最新': 2,
        '最热': 3
    }
    print("请选择排序方式:")
    for idx, order in sort_order.items():
        print(f'{order}. {idx}')
    
    try:
        global selected_order_text
        selected_order_text = input("请输入排序方式对应的名称: ").strip()
        if selected_order_text not in sort_order:
            print("请输入有效的排序方式...")
            return
        
        selected_order_index = sort_order[selected_order_text]
    except Exception as e:
        print(f"处理排序选择时出错: {e}")
        return

    try:
        element = browser.find_element(By.XPATH, '//*[@id="global"]/div[2]/div[2]/div/div[1]/div[2]')
        action.move_to_element(element).perform()# 模拟鼠标悬停
        menu = browser.find_element(By.CLASS_NAME, 'dropdown-items')
        option = menu.find_element(By.XPATH, f'/html/body/div[4]/div/li[{selected_order_index}]')
        option.click()# 模拟鼠标点击

        print('已选择排序方式为:',selected_order_text)
        print('检查时间:',time.ctime())

    except Exception as e:
        print(f"更改排序方式失败: {e}")

change_mode(browser)
change_sort_order(browser, action)

已自动更改模式为图文。
请选择排序方式:
1. 综合
2. 最新
3. 最热
请输入有效的排序方式...


In [6]:
def parsePage(html_content, authorName_list, likeNr_list, URL_list, userURL_list, num):
    """
    解析网页内容并更新数据列表。

    Args:
        html_content (str): 当前页面的HTML内容
        authorName_list (list): 存储作者名字的列表
        likeNr_list (list): 存储获赞数量的列表
        URL_list (list): 存储笔记URL的列表
        userURL_list (list): 存储用户URL的列表
        qbar (tqdm): 进度条对象
        num (int): 需要爬取的笔记数量

    Returns:
        None: 数据存储在传入的列表中
    """
    response = Selector(text=html_content)
    divs = response.xpath('//div[contains(@class, "feeds-container")]/section/div')# 选中网页中包含笔记信息的部分

    # 遍历divs获取每一篇笔记的信息
    for div in divs:
        if len(URL_list) >= num:
            break
        
        if div.xpath('.//span[contains(text(), "大家都在搜")]'):
            continue

        # 选择并提取网页数据
        try:
            author_name = div.xpath('.//a[contains(@class, "author")]/span[contains(@class, "name")]/text()').get()# 作者名字
            like_nr = div.xpath('.//span[contains(@class, "count")]/text()').get()# 获赞数量
            url = div.xpath('.//a[contains(@class, "cover")]/@href').get()# 笔记URL
            user_url = div.xpath('.//a[contains(@class, "author")]/@href').get()# 用户URL
            
            authorName_list.append(author_name)
            likeNr_list.append(like_nr)
            URL_list.append(url)
            userURL_list.append(user_url)

            time.sleep(0.35)
            
        except:
            pass
    
    return True

authorName_list, likeNr_list, URL_list, userURL_list = [], [], [], []
qbar = tqdm(total=num, desc="已获取的笔记数量...")

# 检查是否已经爬取足够数量的笔记，或是否已经达到页面底部
while len(URL_list) < num:
    if '- THE END -' in browser.page_source:
        print(f"当前与{key_word}有关的笔记数量少于 {num}")
        print('检查时间:',time.ctime())
        break
    
    parsePage(browser.page_source, authorName_list, likeNr_list, URL_list, userURL_list, num)
    qbar.update(1)

    if len(URL_list) < num:
        browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')# 模拟鼠标滚动
        time.sleep(random.uniform(3, 5))

if len(URL_list) > num:
    URL_list = URL_list[:num]
    authorName_list = authorName_list[:num]
    likeNr_list = likeNr_list[:num]
    userURL_list = userURL_list[:num]

qbar.close()

已获取的笔记数量...:   6%|▌         | 6/100 [00:54<14:18,  9.13s/it]


In [7]:
if all(len(lst) == num for lst in [authorName_list, likeNr_list, URL_list, userURL_list]):
    print(f"所有属性列表长度均为 {num}，爬取成功！")
    print(f'检查时间: {time.ctime()}')
else:
    min_length = min(map(len, [authorName_list, likeNr_list, URL_list, userURL_list]))
    print(f"当前属性列表长度最小值为 {min_length}，请重新运行上一代码单元，直至所有属性列表长度均为 {num}！")
    print(f'检查时间: {time.ctime()}')

所有属性列表长度均为 100，爬取成功！
检查时间: Wed Dec  4 18:37:33 2024


In [8]:
likeNr_list = extract_large_number(likeNr_list)
URL_list = [re.sub(r'^/search_result/', '/', url) if url is not None else '' for url in URL_list]
userURL_list = [url.split('/')[-1] if url is not None else '' for url in URL_list]

print("以下为清洁数据示例:\n")
for i in range(3):
    print("author_name:", authorName_list[i])
    print("like_nr:", likeNr_list[i])
    print("url:", URL_list[i])
    print("user_url:", userURL_list[i])
    print("------")

以下为清洁数据示例:

author_name: 小红薯63DFCF89
like_nr: 1496
url: /66aa25260000000027012206?xsec_token=ABc2hOAUpWxxUOTRDPNHSViag9ouMHngyKceB-ZH23ACM=&xsec_source=
user_url: 66aa25260000000027012206?xsec_token=ABc2hOAUpWxxUOTRDPNHSViag9ouMHngyKceB-ZH23ACM=&xsec_source=
------
author_name: 卡司特高端留学
like_nr: 728
url: /65e05d7d000000000400320b?xsec_token=ABeiFCyZZMzFmYod2vKcN3x-6hTtIC6tOMluypDdq9tWk=&xsec_source=
user_url: 65e05d7d000000000400320b?xsec_token=ABeiFCyZZMzFmYod2vKcN3x-6hTtIC6tOMluypDdq9tWk=&xsec_source=
------
author_name: 姜姜说留学
like_nr: 51
url: /671f9138000000002401960e?xsec_token=ABN-deT_NSW6qbk2S61WV6ORlsqnPfvAg7nhUjyvIh-1E=&xsec_source=
user_url: 671f9138000000002401960e?xsec_token=ABN-deT_NSW6qbk2S61WV6ORlsqnPfvAg7nhUjyvIh-1E=&xsec_source=
------


In [9]:
def parse_note_page(browser, url, commentNr_list, content_list, datePublished_list, images_list, starNr_list):
    """
    解析单个笔记页面并提取所需数据
    
    Args:
        browser: Selenium WebDriver 实例，用于获取页面内容
        url: 笔记的URL
        commentNr_list (list): 存储评论数量的列表
        content_list (list): 存储笔记内容的列表
        datePublished_list (list): 存储发布时间的列表
        images_list (list): 存储图片链接的列表
        starNr_list (list): 存储收藏数量的列表

    Returns:
        None: 将提取的数据添加到相应的列表中
    """
    whole_url = 'https://www.xiaohongshu.com/explore' + url# 构造完整笔记URL
    browser.get(whole_url)
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@name="description"]')))# 等待页面加载完成
    html = browser.page_source
    selector = Selector(text = html)
    
    try:
        # 选择并提取网页数据
        comment_nr = selector.xpath('//*[@class="total"]/text()').extract_first()# 评论数量
        content = selector.xpath('//*[@name="description"]/@content').extract_first()# 内容
        datePublished = selector.xpath('//*[@class="date"]/text()').extract_first()# 发布时间
        images = selector.xpath('//*[@name="og:image"]/@content').extract_first()# 图片
        images = images + '.jpg' if images else None# 追加图片链接
        star_nr = selector.xpath('//*[@class="count"]/text()').extract_first()# 收藏数量

        commentNr_list.append(comment_nr)
        content_list.append(content)
        datePublished_list.append(datePublished)
        images_list.append(images)
        starNr_list.append(star_nr)

    except:
        pass
    
commentNr_list, content_list, datePublished_list, images_list, starNr_list = [], [], [], [], []

qbar = tqdm(total=len(URL_list), desc="已获取的笔记数量...")
for url in URL_list:
    parse_note_page(browser, url, commentNr_list, content_list, datePublished_list, images_list, starNr_list)
    qbar.update(1)
    time.sleep(random.uniform(0.5, 2))

qbar.close()

已获取的笔记数量...: 100%|██████████| 100/100 [07:20<00:00,  4.41s/it]


In [10]:
if all(len(lst) == num for lst in [commentNr_list, content_list, datePublished_list, images_list, starNr_list]):
    print(f"所有属性列表长度均为 {num}，爬取成功！")
    print(f'检查时间: {time.ctime()}')
else:
    min_length = min(map(len, [commentNr_list, content_list, datePublished_list, images_list, starNr_list]))
    print(f"当前属性列表长度最小值为 {min_length}，请重新运行上一代码单元，直至所有属性列表长度均为 {num}！")
    print(f'检查时间: {time.ctime()}')

所有属性列表长度均为 100，爬取成功！
检查时间: Wed Dec  4 18:45:01 2024


In [11]:
commentNr_list = extract_large_number(extract_number(commentNr_list))
starNr_list = extract_large_number(starNr_list)
datePublished_list = extract_date(datePublished_list)

print("以下为清洁数据示例:\n")
for i in range(3):
    print("comment_nr:", commentNr_list[i])
    print("content:", content_list[i])
    print("datePublished:", datePublished_list[i])
    print("images:", images_list[i])
    print("star_nr:", starNr_list[i])
    print("------")

以下为清洁数据示例:

comment_nr: None
content: 留学商科专业选择，关键在于了解自己的兴趣所在，并找到与之匹配的大方向。这样，选校过程就能更加gao效❗ . 定量组：如果你擅长数学和编程，这些专业可能适合你。 1.金融科技(Fintech)：结合金融知识和计算机技能，是金融行业的新宠。 2.金融工程(Financial Engineering)：金融、数学、物理或工程学背景的你，将进修高等数学和概率论等课程。 3.金融(Finance)：强调定量分析能力，经济学和会计等课程是基础。 4.精算(Actuarial Science)：数学或统计学背景，加上jie决问题的能力，是精算师的必备。 5.信息系统管理(Management Information Systems)：计算机科学或管理学背景，将助你掌握数据库管理和信息系统设计。 6.商业分析(Business Analytics)：统计学和数据分析能力，是商业分析领域的基石。 . 传统商科：如果你对商业运作和管理有兴趣，以下专业可能适合你。 1.管理(Management)：了解基本管理原理和组织行为，是成为优质管理者的起点。 2.会计(Accounting)：商业或会计背景，加上财务管理和审计原理的学习，是会计专业的核心。 . 软专业组：如果你的强项在于沟通和人际交往，这些专业可能更适合你。 1.市场营销(Marketing)：营销和沟通能力，加上市场研究和消费者行为学的学习，是市场营销专业的关键。 2.供应链管理(Supply Chain Management)：物流和运营管理背景，将帮助你掌握项目管理技能。 3.人力资源(Human Resources)：管理、心理学或社会学背景，加上组织行为学和劳动法的学习，是人力资源管理的基础。 4.国际商务(International Business)：商业、管理学或国际关系背景，加上国际贸易和quan球市场营销的学习，是国际商务专业的核心。 . 选择专业时，记得考虑自己的兴趣和背景，这样你的留学之路才能更加顺畅。别忘了，适合自己的才是zui好的！ #商科 #商科留学 #商科硕士 #留学申请季 #留学干货 #留学 #留学申请指南
datePublished: 2024-07-31
images: http://sns-webpic-qc.xh

In [12]:
def parse_author_page(browser, user_url, authorCollectNr_list, authorFansNr_list, authorNoteNr_list):
    """
    解析单个用户页面并提取所需数据
    
    Args:
        browser: Selenium WebDriver 实例，用于获取页面内容
        user_url: 用户的URL
        authorCollectNr_list (list): 存储作者获赞与收藏数量的列表
        authorFansNr_list (list): 存储作者粉丝数量的列表
        authorNoteNr_list (list): 存储作者笔记数量的列表
    
    Returns:
        None: 将提取的数据添加到相应的列表中
    """

    whole_url = 'https://www.xiaohongshu.com/user/profile/' + user_url# 构造完整用户页面URL
    browser.get(whole_url)

    # 模拟滚动页面直到加载完成
    while True:
        previous_page_height = browser.execute_script("return document.body.scrollHeight")
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")# 滚动到页面底部
        time.sleep(random.uniform(1, 2))

        current_page_height = browser.execute_script("return document.body.scrollHeight")
        if current_page_height == previous_page_height:
            break
        previous_page_height = current_page_height

    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="userPostedFeeds"]//section')))# 等待页面加载完成
    html = browser.page_source
    selector = Selector(text=html)

    try:
        # 选择并提取网页数据
        author_collect_nr = selector.xpath('//*[@class="data-info"]/div[1]/div[3]/span[@class="count"]/text()').extract_first()# 作者获赞与收藏数量
        author_fans_nr = selector.xpath('//*[@class="data-info"]/div[1]/div[2]/span[@class="count"]/text()').extract_first()# 作者粉丝数量
        author_note_nr = len(selector.xpath('//*[@id="userPostedFeeds"]//section'))# 作者笔记数量

        authorCollectNr_list.append(author_collect_nr)
        authorFansNr_list.append(author_fans_nr)
        authorNoteNr_list.append(author_note_nr)

    except Exception as e:
        pass

authorCollectNr_list, authorFansNr_list, authorNoteNr_list = [], [], []

qbar = tqdm(total=len(userURL_list), desc="已爬取的作者数量...")
for user_url in userURL_list:
    parse_author_page(browser, user_url, authorCollectNr_list, authorFansNr_list, authorNoteNr_list)
    qbar.update(1)
    time.sleep(random.uniform(1, 2))

qbar.close()

已爬取的作者数量...:   0%|          | 0/100 [00:00<?, ?it/s]

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000105331548 cxxbridge1$str$ptr + 3618920
1   chromedriver                        0x0000000105329d98 cxxbridge1$str$ptr + 3588280
2   chromedriver                        0x0000000104d989c4 cxxbridge1$string$len + 89228
3   chromedriver                        0x0000000104ddcda8 cxxbridge1$string$len + 368752
4   chromedriver                        0x0000000104e1654c cxxbridge1$string$len + 604180
5   chromedriver                        0x0000000104dd15c0 cxxbridge1$string$len + 321672
6   chromedriver                        0x0000000104dd2210 cxxbridge1$string$len + 324824
7   chromedriver                        0x00000001052fd4bc cxxbridge1$str$ptr + 3405788
8   chromedriver                        0x00000001053007dc cxxbridge1$str$ptr + 3418876
9   chromedriver                        0x00000001052e4130 cxxbridge1$str$ptr + 3302480
10  chromedriver                        0x000000010530109c cxxbridge1$s

In [13]:
if all(len(lst) == num for lst in [authorCollectNr_list, authorFansNr_list, authorNoteNr_list]):
    print(f"所有属性列表长度均为 {num}，爬取成功！")
    print(f'检查时间: {time.ctime()}')
else:
    min_length = min(map(len, [authorCollectNr_list, authorFansNr_list, authorNoteNr_list]))
    print(f"当前属性列表长度最小值为 {min_length}，请重新运行上一代码单元，直至所有属性列表长度均为 {num}！")
    print(f'检查时间: {time.ctime()}')

当前属性列表长度最小值为 0，请重新运行上一代码单元，直至所有属性列表长度均为 100！
检查时间: Wed Dec  4 18:45:28 2024


In [14]:
authorCollectNr_list = extract_large_number(authorCollectNr_list)
authorFansNr_list = extract_large_number(authorFansNr_list)

print("以下为清洁数据示例：")
for i in range(3):
    print("author_collect_nr:", authorCollectNr_list[i])
    print("author_fans_nr:", authorFansNr_list[i])
    print("author_note_nr:", authorNoteNr_list[i])
    print("------")

以下为清洁数据示例：


IndexError: list index out of range

In [15]:
dic={
    # "author_collect_nr": authorCollectNr_list,# 作者获赞与收藏数量
    # "author_fans_nr": authorFansNr_list,# 粉丝数量
    "author_name": authorName_list,# 作者名字
    # "author_note_nr": authorNoteNr_list,# 作者笔记数量
    "comment_nr": commentNr_list,# 笔记评论数量
    "content": content_list,# 笔记内容
    "datePublished": datePublished_list,# 笔记发布日期
    "images": images_list,# 笔记封面图片
    "like_nr": likeNr_list,# 笔记获赞数量
    "star_nr" : starNr_list,
    "url": URL_list,# 笔记URL
    "user_url": userURL_list# 作者URL
    }

df = pd.DataFrame.from_dict(dic)
df = df[~df.duplicated(keep='first')]# 检索并删除所有属性值都相同的行,即保留第一次出现的行，删除后续的重复行
print("删除", num-len(df), "行重复行后剩余", len(df), "行。")
print('检查时间:',time.ctime())
display(df.head(10))
df.to_csv('scraped_data.csv', encoding='utf-8-sig')
print("数据已保存到 'scraped_data.csv'")

删除 10 行重复行后剩余 90 行。
检查时间: Wed Dec  4 18:45:32 2024


,author_name,comment_nr,content,datePublished,images,like_nr,star_nr,url,user_url
0,小红薯63DFCF89,NaN,留学商科专业选择，关键在于了解自己的兴趣所在，并找到与之匹配的大方向。这样，选校过程就能更加...,2024-07-31,http://sns-webpic-qc.xhscdn.com/202412050737/a...,1496.0,1,/66aa25260000000027012206?xsec_token=ABc2hOAUp...,66aa25260000000027012206?xsec_token=ABc2hOAUpW...
1,卡司特高端留学,57.0,2024QS商科硕士专业排名已出炉，摘录了金融、管理、商业分析、市场营销四大专业，给大家简单...,2024-03-20,http://sns-webpic-qc.xhscdn.com/202412050737/4...,728.0,1,/65e05d7d000000000400320b?xsec_token=ABeiFCyZZ...,65e05d7d000000000400320b?xsec_token=ABeiFCyZZM...
2,姜姜说留学,6.0,根据美国劳工统计局的数据，STEM专业带动的工作岗位，其增长速度和薪酬水平均在美国就业市场处...,2024-10-28,http://sns-webpic-qc.xhscdn.com/202412050737/e...,51.0,1,/671f9138000000002401960e?xsec_token=ABN-deT_N...,671f9138000000002401960e?xsec_token=ABN-deT_NS...
3,Chloe在留学,22.0,🍭因为商科还有些可投递，我就寻思把这个系列给大家持续更出来，最大限度的帮同学们节省时间，选专...,2024-02-14,http://sns-webpic-qc.xhscdn.com/202412050737/d...,680.0,1,/65ccc5c0000000002d0009cd?xsec_token=ABNS3Sab_...,65ccc5c0000000002d0009cd?xsec_token=ABNS3Sab_K...
4,芋泥学姐,17.0,今天为大家整理了QS2025商科管理专业TOP50榜单，一起来看看吧！ - 👑斯坦福大学商学...,2024-10-26,http://sns-webpic-qc.xhscdn.com/202412050737/5...,128.0,1,/671cd97a000000002401a003?xsec_token=AB0fnW-Eu...,671cd97a000000002401a003?xsec_token=AB0fnW-Euy...
5,Allen的留学情报站,NaN,#留学美国 #美国研究生 #美国研究生申请 #美国研究生留学申请,2024-07-10,http://sns-webpic-qc.xhscdn.com/202412050737/e...,36.0,1,/668e095e00000000250035b3?xsec_token=ABOiJSK6o...,668e095e00000000250035b3?xsec_token=ABOiJSK6o_...
6,Winnie学姐的英国日记,5.0,10个留学生8个学商科！ \t 好家伙，🇬🇧英国你直接点名中国学生得了。作为中国学生“扎堆”...,2024-11-07,http://sns-webpic-qc.xhscdn.com/202412050738/e...,31.0,1,/672caf99000000003c01b666?xsec_token=ABi-WlgaB...,672caf99000000003c01b666?xsec_token=ABi-WlgaBB...
7,Rachel学姐🐙,NaN,🌟想读商科又想进名校，但背景不够强？来看看美国名校的泛商科水项目吧！低绩点的同学也有机会冲哦...,2024-10-10,http://sns-webpic-qc.xhscdn.com/202412050738/a...,87.0,1,/66bf28200000000005031860?xsec_token=ABT_ZUM5v...,66bf28200000000005031860?xsec_token=ABT_ZUM5vV...
8,viki学姐硕博留学,NaN,🧮会计学是美国留学的热门选择，作为美国商学院的传统专业方向之一，一直保持着较为稳定的就业需求...,2024-09-28,http://sns-webpic-qc.xhscdn.com/202412050738/4...,47.0,1,/66f7ddb2000000002c02c8ee?xsec_token=ABtMnnPjX...,66f7ddb2000000002c02c8ee?xsec_token=ABtMnnPjXa...
9,Cheese学姐留学指南,36.0,留学的热门专业里，金融类几乎可以算是第一，不少本科学其他类型专业的学生，也会通过在研究生阶段...,2024-03-27,http://sns-webpic-qc.xhscdn.com/202412050738/8...,214.0,1,/6603f2b10000000012033c24?xsec_token=ABALokOeF...,6603f2b10000000012033c24?xsec_token=ABALokOeF0...


数据已保存到 'scraped_data.csv'
